<a href="https://colab.research.google.com/github/hafizhahnajwa20/SISTEM-REKOMENDASI-ABON-UMI/blob/main/Copy_of_Sistm_rekomendasi_Abon_umi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. MOUNT GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. IMPORT LIBRARY

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

3. BACA DATASET DARI GOOGLE DRIVE

In [ ]:
file_path = "/content/drive/MyDrive/abon.csv"
df = pd.read_csv(file_path)


4. PRA-PROSES DATA
Konversi Harga Satuan dan Total ke numerik

In [ ]:
df["Harga Satuan"] = df["Harga Satuan"].str.replace("Rp", "").str.replace(".", "").str.replace(",00", "").astype(int)
df["Total"] = df["Total"].str.replace("Rp", "").str.replace(".", "").str.replace(",00", "").astype(int)

Ekstrak bulan dan hari dari Tanggal

In [ ]:
df["Tanggal"] = pd.to_datetime(df["Tanggal"], dayfirst=True)
df["Bulan"] = df["Tanggal"].dt.month
df["Hari"] = df["Tanggal"].dt.dayofweek  # 0 = Senin, 6 = Minggu

Encode fitur kategori menjadi angka

In [ ]:
encoder_nama = LabelEncoder()
encoder_jenis = LabelEncoder()
encoder_berat = LabelEncoder()
df["Nama Abon"] = encoder_nama.fit_transform(df["Nama Abon"])
df["Jenis Abon"] = encoder_jenis.fit_transform(df["Jenis Abon"])
df["Berat Kemasan"] = encoder_berat.fit_transform(df["Berat Kemasan"])

 5. PERSIAPKAN FITUR (X) DAN TARGET (y)

In [ ]:
X = df[["Nama Abon", "Jenis Abon", "Berat Kemasan", "Harga Satuan", "Bulan", "Hari"]]
y = df["Jumlah"]

6. SPLIT DATA TRAIN/TEST

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

7. LATIH MODEL NAIVE BAYES

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

 8. PREDIKSI DAN EVALUASI

In [ ]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R2 Score: {r2:.2f}")


📌 Mean Squared Error (MSE): 127.28
📌 R2 Score: -8.26


9. CONTOH PREDIKSI BARU

In [ ]:
# Misalnya: Abon Ikan, Pedas, 100 gram, harga 22000, bulan 6, hari Senin(0)
sample = np.array([[encoder_nama.transform(["Abon Ikan"])[0],
                    encoder_jenis.transform(["Pedas"])[0],
                    encoder_berat.transform(["100 gram"])[0],
                    22000, 6, 0]])

prediksi_stok = model.predict(sample)
print(f"🔮 Prediksi Jumlah Stok: {prediksi_stok[0]}")

🔮 Prediksi Jumlah Stok: 1


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [ ]:
# Analisis jumlah permintaan per jenis abon
analisis = df.groupby(["Nama Abon", "Jenis Abon", "Berat Kemasan"])["Jumlah"].sum().reset_index()

In [ ]:
# Cari varian dengan permintaan tertinggi
top_varian = analisis.sort_values(by="Jumlah", ascending=False).head(5)
print("📊 Top 5 Varian Abon dengan Permintaan Tertinggi:")
print(top_varian)

📊 Top 5 Varian Abon dengan Permintaan Tertinggi:
    Nama Abon  Jenis Abon  Berat Kemasan  Jumlah
0           0           0              0     237
6           1           0              0     220
1           0           0              1     208
9           1           1              0     203
15          2           1              0     166


In [ ]:
# Kesimpulan varian dengan stok terbanyak
varian_tertinggi = top_varian.iloc[0]
print(f"\n✅ Kesimpulan: Varian yang harus disiapkan stok lebih banyak adalah "
      f"{encoder_nama.inverse_transform([varian_tertinggi['Nama Abon']])[0]} "
      f"{encoder_jenis.inverse_transform([varian_tertinggi['Jenis Abon']])[0]} "
      f"{encoder_berat.inverse_transform([varian_tertinggi['Berat Kemasan']])[0]} "
      f"dengan total permintaan {varian_tertinggi['Jumlah']} pcs.")


✅ Kesimpulan: Varian yang harus disiapkan stok lebih banyak adalah Abon Ayam Manis 100 gram dengan total permintaan 237 pcs.


In [ ]:
# === 1. Analisis permintaan historis ===
analisis = df.groupby(["Nama Abon", "Jenis Abon", "Berat Kemasan"])["Jumlah"].sum().reset_index()

# Urutkan berdasarkan permintaan tertinggi
top_varian = analisis.sort_values(by="Jumlah", ascending=False)
print("📊 Top Varian Abon Berdasarkan Data Historis:")
print(top_varian.head(5))

# === 2. Ambil varian dengan permintaan tertinggi ===
varian_tertinggi = top_varian.iloc[0]

# Dekode nama, jenis, dan berat dari angka ke label
nama_tertinggi = encoder_nama.inverse_transform([varian_tertinggi["Nama Abon"]])[0]
jenis_tertinggi = encoder_jenis.inverse_transform([varian_tertinggi["Jenis Abon"]])[0]
berat_tertinggi = encoder_berat.inverse_transform([varian_tertinggi["Berat Kemasan"]])[0]

# === 3. Kesimpulan Rekomendasi Ready Stock ===
print("\n✅ Rekomendasi Stok Utama:")
print(f"Varian **{nama_tertinggi} {jenis_tertinggi} {berat_tertinggi}** "
      f"perlu disiapkan stok lebih banyak dengan total permintaan historis "
      f"{varian_tertinggi['Jumlah']} pcs.")

# === 4. (Opsional) Prediksi stok ke depan untuk varian ini ===
sample = np.array([[varian_tertinggi["Nama Abon"],
                    varian_tertinggi["Jenis Abon"],
                    varian_tertinggi["Berat Kemasan"],
                    22000, 8, 2]])  # Contoh: harga 22000, bulan 8, hari Rabu(2)
prediksi_stok = model.predict(sample)
print(f"🔮 Prediksi kebutuhan stok ke depan: {prediksi_stok[0]} pcs.")

📊 Top Varian Abon Berdasarkan Data Historis:
    Nama Abon  Jenis Abon  Berat Kemasan  Jumlah
0           0           0              0     237
6           1           0              0     220
1           0           0              1     208
9           1           1              0     203
15          2           1              0     166

✅ Rekomendasi Stok Utama:
Varian **Abon Ayam Manis 100 gram** perlu disiapkan stok lebih banyak dengan total permintaan historis 237 pcs.
🔮 Prediksi kebutuhan stok ke depan: 1 pcs.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
